# Real-Time MRI with BART 🎥

## Structure

### Part 1: Simple Real-Time MRI Reconstruction

Using the adjoint NuFFT paired with the Ram-Lak Filter, a real-time dataset
acquired with a turn-based radial FLASH sequence is reconstructed.

### Part 2: Iterative Reconstruction

We transform the previous reconstruction into something more modern.

### Part 3: iGRASP and Real-Time NLINV

We use BART to run the well-known iGRASP method.

A previously "hidden" step, the creation of coil sensitivities, is examined further. We look at a different iterative reconstruction scheme which is very suitable for real-time MRI and real-time reconstruction.

## Setup Environment for BART

This notebook assumes bart is 'available', i.e. has been successfully installed beforehand.


If this doesn't work, please consult the [installation guide](https://mrirecon.github.io/bart/installation.html).

✅ You should be able to run bart "directly" in this notebook:

In [ ]:
! bart version

# Part 3: iGRASP and Real-Time NLINV

In this exercise, we will
- Change to another dataset, acquired with golden angle
- Add  TV regularization, that is, add the term $||\frac{d}{dt}  x||_1$ to the optimization problem solved by pics.

In combination, this is the well-known "iGRASP" method.

## Reading in the data

- Change into a new directory for this exercise, as we're working with a different dataset

In [ ]:
! mkdir -p session3_2
%cd session3_2

Download the dataset.

This might take a while, it is slighly larger.

In [ ]:
%%bash
[ -f ksp_gi7.ra ] || wget https://cloud.tugraz.at/index.php/s/YtJLSmdwfDZrXzX/download/ksp_gi7.ra
bart copy ksp_gi7.ra ksp

## 'Publish' data parameters & Rebinning choice

This cell should be used to set the parameters for the following commands using "environment variables".

As we can't do this persistently from `%%bash` cells, it is done in Python.

✏️ **Adjust the parameters to the actually measured file**

In [ ]:
# measurement (or simulation) parameters
spokes = 3920
golden_index = 7
baseres = 196

# rebinning choice:
spokes_per_frame = 15


# calculate rebinned dataset
frames = (spokes // spokes_per_frame)
spokes_cropped = spokes_per_frame * frames

# export
import os
os.environ["SPOKES"] = str(spokes)
os.environ["SPOKES_PER_FRAME"] = str(spokes_per_frame)
os.environ["SPOKES_CROPPED"] = str(spokes_cropped)
os.environ["FRAMES"] = str(frames)
os.environ["GI"] = str(golden_index)
os.environ["BASERES"] = str(baseres)

---

## Alternative: Simulate a golden angle dataset

If no raw data is available, you can use the `bart phantom` tool to generate a dataset.

Unfortunately, it doesn't move, but it can still help quickly testing your algorithms!

In [ ]:
%%bash
# might take a few seconds, depending on the number of coils!
set -eu

NCOILS=4

bart traj -r -D -o2 -x$BASERES -y$SPOKES -G -s$GI trj
bart phantom -t trj -k -s$NCOILS ksp

## Data preprocessing and rebinning

We start with coil compression as per the usual:

In [ ]:
%%bash
bart cc -p8 ksp ksp_cc

The next step (and first task) is to re-bin the dataset.
Previously, we already exported the environment variables FRAMES, SPOKES_PER_FRAME, and SPOKES_CROPPED = FRAMES * SPOKES_PER_FRAME.

✏️ **Re-Binning**:
- Resize the dataset to SPOKES_CROPPED spokes, removing the last few spokes
- Reshape the dataset, so that we get a file with several frames and a number of spokes per frame, instead of one big frame.

output filename: ksp_frames

In [ ]:
%%bash


<details>
    <summary>Show solution</summary>

    %%bash
    bart resize 2 $SPOKES_CROPPED ksp_cc ksp_cropped
    bart reshape $(bart bitmask 2 10) $SPOKES_PER_FRAME $FRAMES ksp_cropped ksp_frames
</details>

✅ The output file should now have length FRAMES along dimension 10 and length SPOKES_PER_FRAME along dim 2.

You can check with the following code:

In [ ]:
%%bash

[ $FRAMES -eq $(bart show -d 10 ksp_frames) ] && \
[ $SPOKES_PER_FRAME -eq $(bart show -d 2 ksp_frames) ] && echo "Looks good!" || echo "Something's not yet right."

bart show -m ksp_frames

## Generating the trajectory

✏️ **Generate a trajectory that matches the raw data.**


The output file should be named "trj".

<details>
    <summary> Show me the parameters to the traj command </summary>

    `bart traj -r -D -G -o2 -x$BASERES -y$SPOKES -G -s$GI trj`
</details>

In [ ]:
%%bash
bart traj -r

<details>
    <summary>Show solution</summary>
    
    %%bash
    bart traj -r -D -o2 -x$BASERES -y$SPOKES -G -s$GI trj
</details>

✏️ **Rebin the trajectory**
- Now, apply the same steps previously applied to the data to re-bin the trajectory:

In [ ]:
%%bash


<details>
    <summary>Show solution</summary>
    
    %%bash
    # rebin traj
    bart resize 2 $SPOKES_CROPPED trj trj_cropped
    bart reshape $(bart bitmask 2 10) $SPOKES_PER_FRAME $FRAMES trj_cropped trj_frames
</details>

## Estimating coil sensitivities

To estimate coil sensitivities, we'll proceed as before, this time using the complete non-rebinned dataset as an input 

In [ ]:
%%bash
# full frame for coil estimation
bart scale 1.5 trj trj_os
bart ncalib -t trj_os ksp_cc coils

## iGRASP Reco

and finally!

✏️ **Run an iGRASP Reco using BART PICS**

Some hints:

- `pics -Rh` gives you information about the available regularization options. Look for `T`
- The "joint" threshold flags are zero.
- A good regularization starting value is ~0.01
- Remember scaling the trajectory! The value used here must match the value used before to reconstruct the coils.
- Start with a few frames instead of the whole dataset, to see the command is working as intended.

In [ ]:
%%bash


<details>
    <summary>Show solution</summary>
    
    %%bash
    
    bart extract 10 0 10 ksp_frames ksp_small
    bart extract 10 0 10 trj_frames trj_small
    
    bart scale 1.5 trj_small trj_small_os
    
    bart pics -t trj_small_os ksp_small coils img_pics
    
    bart pics -RT:1024:0:0.01 -t trj_small_os ksp_small coils img_pics_os
    
    bart resize -c 0 $BASERES 1 $BASERES img_pics_os img_pics
</details>

In [ ]:
!bart view img_pics

### Parameter exploration

Spokes per frame as well as regularization strength were chosen more or less arbitrarily here.

Generally speaking, a low regularization strength and a low number of spokes per frame are desirable, but there is a trade-off.

✏️ **Try to find parameter values which lead to (subjectively) optimal spatio-temporal resolution**

## Real-Time NLINV

So far, we've been using the elusive `bart ncalib` command to estimate sensitivities for our reconstructions.

ncalib under the hood uses nonlinear regularized inversion (NLINV), a method which estimates coil sensitivities and images in a single optimization step. The images are 'thrown away' in ncalib, albeit the method is very well-suited for real-time MRI.

Thus, in this last part, we will use NLINV for image reconstruction without prior calibration step.

✏️ **Reconstruct the iGRASP dataset with Real-Time NLINV**

<details>
    <summary>Hint about rtnlinv vs nlinv</summary>

there are currently two nlinv commands in BART, nlinv and rtnlinv.
If you built from source, we recommend that you use `nlinv --real-time` instead of rtnlinv.
However, if you are stuck on BART v0.9.00, `rtnlinv` should work as well, but it is much slower.
</details>

In [ ]:
%%bash


<details>
    <summary>Show solution</summary>

    %%bash
    bart extract 10 0 10 ksp_frames ksp_small
    bart extract 10 0 10 trj_frames trj_small
    
    bart scale 1.5 trj_small trj_small_os
    
    # for BART 0.9.00:
    # bart rtnlinv -t trj_small_os ksp_small img_nlinv_os
    bart nlinv --real-time -t trj_small_os ksp_small img_nlinv_os
    
    bart resize -c 0 $BASERES 1 $BASERES img_nlinv_os img_nlinv
</details>

In [ ]:
!bart view img_nlinv

---

This is the end of this tutorial on real-time MRI. :)
🏁🐐🎉 All done!

Hope you enjoyed!